In [ ]:
import gymnasium as gym
import numpy as np

from scipy.spatial import ConvexHull

env = gym.make("Ant-v4", render_mode="human")  # replace with HalfCheetah-v4 or Walker2d-v4
obs = env.reset()

for _ in range(1000):
    env.render()
    action = env.action_space.sample()  # random actions
    obs, reward, terminated, truncated, info = env.step(action)

    done = terminated or truncated
    if done:
        obs = env.reset()

env.close()


/opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(
/opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [2]:

env = gym.make("Ant-v4", render_mode="human")
obs, _ = env.reset()

for _ in range(1000):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

    sim = env.unwrapped.data
    model = env.unwrapped.model

    # --- Center of Mass position ---
    masses = model.body_mass              # (num_bodies,)
    xpos = sim.xipos                      # (num_bodies, 3), body CoM positions
    com = np.sum(masses[:, None] * xpos, axis=0) / np.sum(masses)

    # --- Center of Mass linear velocity ---
    # Option 1: Use subtree_linvel (available in newer mujoco)
    vels = sim.subtree_linvel             # (num_bodies, 3), linear velocity of each subtree’s CoM
    com_vel = np.sum(masses[:, None] * vels, axis=0) / np.sum(masses)

    print("CoM:", com, "   CoM Velocity:", com_vel)

    if terminated or truncated:
        obs, _ = env.reset()

env.close()


CoM: [0.09435219 0.0025763  0.6772742 ]    CoM Velocity: [0. 0. 0.]
CoM: [ 0.09737096 -0.00390476  0.63743194]    CoM Velocity: [0. 0. 0.]
CoM: [ 0.10024203 -0.01269713  0.58266065]    CoM Velocity: [0. 0. 0.]
CoM: [ 0.09977957 -0.01900937  0.55878241]    CoM Velocity: [0. 0. 0.]
CoM: [0.10554202 0.00711114 0.57593483]    CoM Velocity: [0. 0. 0.]
CoM: [0.11219171 0.03330185 0.56912965]    CoM Velocity: [0. 0. 0.]
CoM: [0.11412321 0.05969079 0.54408416]    CoM Velocity: [0. 0. 0.]
CoM: [0.10343997 0.08565545 0.52361392]    CoM Velocity: [0. 0. 0.]
CoM: [0.09838522 0.10229688 0.54879688]    CoM Velocity: [0. 0. 0.]
CoM: [0.09330526 0.11347021 0.56287825]    CoM Velocity: [0. 0. 0.]
CoM: [0.07691484 0.11082827 0.57749309]    CoM Velocity: [0. 0. 0.]
CoM: [0.06050802 0.10831667 0.56750158]    CoM Velocity: [0. 0. 0.]
CoM: [0.04722947 0.11685407 0.54877408]    CoM Velocity: [0. 0. 0.]
CoM: [0.03456268 0.12387932 0.51055699]    CoM Velocity: [0. 0. 0.]
CoM: [0.02533079 0.13018598 0.45445803]

In [3]:
def get_geom_name(model, geom_id):
    return mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_GEOM, geom_id)

def stability_margin(sim, model):
    # --- Step 1: Get CoM ---
    masses = model.body_mass
    xpos = sim.xipos
    com = np.sum(masses[:, None] * xpos, axis=0) / np.sum(masses)
    com_xy = com[:2]  # project to XY plane

    # --- Step 2: Get foot contacts ---
    contact_points = []
    for i in range(sim.ncon):  # loop over contacts
        contact = sim.contact[i]
        geom1, geom2 = contact.geom1, contact.geom2
        # Identify if one of geoms is a foot (by name)
        for geom_id in [geom1, geom2]:
            geom_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_GEOM, geom_id)
            if geom_name and "foot" in geom_name:
                pos = sim.geom_xpos[geom_id][:2]  # XY only
                contact_points.append(pos)
    contact_points = np.unique(contact_points, axis=0)

    if len(contact_points) < 3:
        return None  # can't form polygon (unstable)

    # --- Step 3: Build convex hull ---
    hull = ConvexHull(contact_points)
    hull_points = contact_points[hull.vertices]

    # --- Step 4: Compute distance to hull ---
    # Simple check: inside hull or not
    from shapely.geometry import Point, Polygon
    poly = Polygon(hull_points)
    point = Point(com_xy)
    if poly.contains(point):
        margin = point.distance(poly.exterior)
    else:
        margin = -point.distance(poly.exterior)
    return margin

# -------------------
# Run example
env = gym.make("Ant-v4", render_mode="human")
obs, _ = env.reset()

for _ in range(500):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

    sim, model = env.unwrapped.data, env.unwrapped.model
    margin = stability_margin(sim, model)
    if margin is not None:
        print("Stability margin:", margin)

    if terminated or truncated:
        obs, _ = env.reset()

env.close()


/opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)
